<a href="https://colab.research.google.com/github/t-willi/Simula/blob/main/Feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
!pip install ecg_plot
!pip install wfdb

In [ ]:
import pathlib
from pathlib import Path
# current_position=pathlib.PurePath(__file__)
# dir=current_position.parent
# main_folder=dir.joinpath("main_folder")
current_position=pathlib.PurePath("/content/")
main_folder=current_position.joinpath("main_folder")
Path(main_folder).mkdir(parents=False,exist_ok = True)
#create folder and directory for artifacts
artifact_dir=main_folder.joinpath("artifacts")
Path(artifact_dir).mkdir(parents=False,exist_ok = True)
#create folder and directory for train_data
train_dir=main_folder.joinpath("train_dir")
Path(train_dir).mkdir(parents=False,exist_ok = True)
#create folder and directory for ecg_files
ecg_dir=main_folder.joinpath("ecg")
Path(ecg_dir).mkdir(parents=False,exist_ok = True)
#create folder and directory for saved model sate dicts
model_dir=main_folder.joinpath("model")
Path(model_dir).mkdir(parents=False,exist_ok = True)


In [ ]:
import wandb
import pandas as pd
import torch
from torch import nn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import torch.optim as optim
from random import shuffle
from tqdm.auto import tqdm
import requests
import zipfile
from pathlib import Path

if torch.cuda.is_available()==True:
  device="cuda:0"
else:
  device ="cpu"

apikey = "7a8ee9d41cc2d51eb77fd795e14f74a215e63c2d"
api = wandb.Api()
artifact_syn = api.artifact('ecg_simula/setup_weights and biases/ecg_25000.zip:v0')
artifact_normal = api.artifact('ecg_simula/upload_PTB_XL_normal.zip/PTB_normal:v0', type='dataset')
artifact_syn.download()
artifact_normal.download()
pTOP_7_7_syn = api.artifact('ecg_simula/Feature_extraction_7to7_syn/Model:v4', type='Model')
pTOP_7_7_norm = api.artifact('ecg_simula/Feature_extraction_7to7_PTB/Model:v6', type='Model')
pTOP_1_7_syn = api.artifact('ecg_simula/AE_pTOP_serverrun_synthetic_data/Model:v6', type='Model')
## pTOP_1_7_norm comes from training of pTOP on PTB_normal data with pretraining from ecg_simula/AE_pTOP_serverrun_synthetic_data/Model:v3
pTOP_1_7_norm = api.artifact('ecg_simula/pTOP_PTB_normal/Model:v54', type='Model')
## download model states 
pTOP_7_7_syn.download("/content/artifacts/pTOP_7_7_syn")
pTOP_7_7_norm.download("/content/artifacts/pTOP_7_7_norm")
pTOP_1_7_syn.download("/content/artifacts/pTOP_1_7_syn")
pTOP_1_7_norm.download("/content/artifacts/pTOP_1_7_norm")

torch.manual_seed(42)

In [ ]:
# previous_model = api.artifact('ecg_simula/AE_pTOP_LS_optim/Model:v8', type='Model')
# previous_model.download()

def request(path=None,name=None):
  import requests
  from pathlib import Path
  request = requests.get(path)
  name=name+".py"
  with open(name,"wb") as f:
    f.write(request.content)

unzip_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/unzip.py"
get_pred_no_reshape_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/get_predictions_no_reshape.py"
get_pred_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/get_predictions.py"
plt_ECG_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/plot_ECG.py"
Dataloader_syn_1to7_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/dataloader_syn_1to7.py"
Dataloader_syn_7to7_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/dataloader_syn_7to7.py"
Dataloader_normal_1to7_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/dataloader_PTB_normal_1to7.py"
Dataloader_normal_7to7_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/dataloader_PTB_normal_7to7.py"
pTOP_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/pTOP_model.py"

request(unzip_git_dir,"Unzip")
from Unzip import unzip

#request(pTOP_git_dir,"pTOP")
#from pTOP import *
request(Dataloader_syn_1to7_git_dir,"dataset_and_loader")
#from dataset_and_loader import Custom_dataset as CD
from dataset_and_loader import make_loader as ml
request(Dataloader_syn_1to7_git_dir,"syn1_7")
from syn1_7 import Custom_dataset as CD_syn_1_7
request(Dataloader_syn_7to7_git_dir,"syn7_7")
from syn7_7 import Custom_dataset as CD_syn_7_7
request(Dataloader_normal_1to7_git_dir,"normal1_7")
from normal1_7 import Custom_dataset_PTB as CD_normal_1_7
request(Dataloader_normal_7to7_git_dir,"normal7_7")
from normal7_7 import Custom_dataset_PTB as CD_normal_7_7


#download prediction generator
request(get_pred_no_reshape_git_dir,"get_predictions")
from get_predictions import get_pred
#download ECG plotter
request(plt_ECG_git_dir,"plot_ECG")
from plot_ECG import plotECG

unzip(save_path="custom_data/syn_train",zip_path="/content/artifacts/ecg_25000.zip:v0/ecg_25000.zip",reload=True)
unzip(save_path="custom_data/norm_train",zip_path="/content/artifacts/PTB_normal:v0/PTB_normal.zip",reload=True)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

class Transpose1dLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding=11, upsample=None, output_padding=1):
        super(Transpose1dLayer, self).__init__()
        self.upsample = upsample
        self.upsample_layer = torch.nn.Upsample(scale_factor=upsample)
        reflection_pad = kernel_size // 2
        self.reflection_pad = nn.ConstantPad1d(reflection_pad, value=0)
        self.conv1d = torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride)
        self.Conv1dTrans = nn.ConvTranspose1d(in_channels, out_channels, kernel_size, stride, padding, output_padding)

    def forward(self, x):
        if self.upsample:
            #x = torch.cat((x, in_feature), 1)
            return self.conv1d(self.reflection_pad(self.upsample_layer(x)))
        else:
            return self.Conv1dTrans(x)


class pTOP_7to7(nn.Module):
    def __init__(self,latent_dim=100, post_proc_filt_len=512,upsample=True):
        super(pTOP_7to7, self).__init__()
        # "Dense" is the same meaning as fully connection.
        stride = 4
        if upsample:
            stride = 1
            upsample = 5
        # if upsample is anything but none Transpose1dLayer will do
        # self.conv1d(self.reflection_pad(self.upsample_layer(x)))
        # which is a 1d convolution on padded and upsampled data x
        self.deconv_1 = Transpose1dLayer(250 , 250, 25, stride, upsample=upsample)
        self.deconv_2 = Transpose1dLayer(250, 150, 25, stride, upsample=upsample)
        self.deconv_3 = Transpose1dLayer(150, 50, 25, stride, upsample=upsample)
        self.deconv_4 = Transpose1dLayer( 50, 25, 25, stride, upsample=2)
        self.deconv_5 = Transpose1dLayer( 25, 10, 25, stride, upsample=upsample)
        self.deconv_6 = Transpose1dLayer(  10, 7, 25, stride, upsample=2)


        #new convolutional layers
        self.conv_1 = nn.Conv1d(7, 10, 25, stride=2, padding=25 // 2)
        self.conv_2 = nn.Conv1d(10, 25, 25, stride=5, padding= 25 // 2)
        self.conv_3 = nn.Conv1d(25, 50 , 25, stride=2, padding= 25 // 2)
        self.conv_4 = nn.Conv1d(50, 150 , 25, stride=5, padding= 25 // 2)
        self.conv_5 = nn.Conv1d(150, 250 , 25, stride=5, padding= 25 // 2)
        self.conv_6 = nn.Conv1d(250, 250 , 25, stride=5, padding= 25 // 2)
        self.flatt = nn.Flatten()
        self.linear1 = nn.Linear(500,100)
        self.linear2 = nn.Linear(100,500)

        for m in self.modules():
            if isinstance(m, nn.ConvTranspose1d) or isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight.data)

    def forward(self, x, LS=False):
        self.LS=LS
        if x.ndim==2:
          x=x.unsqueeze(0)
        x = F.leaky_relu(self.conv_1(x)) #(1,7,5000 --> 1, 10, 2500)
        x = F.leaky_relu(self.conv_2(x)) #( --> 1, 25, 500)
        x = F.leaky_relu(self.conv_3(x)) #(--> 1, 50, 250)
        x = F.leaky_relu(self.conv_4(x)) # --> 1, 150, 50)
        x = F.leaky_relu(self.conv_5(x)) #(--> 1, 250, 10)
        x = F.leaky_relu(self.conv_6(x)) #(--> 1, 250, 2)-->flatten into (1,500)), then to linear ((1,100)), and then back
        x = self.flatt(x) # (1,500)
        LS = self.linear1(x) #(1,100)
        if self.LS is True:
          return LS
        x = self.linear2(LS) #(1,500)
        zero_dim=x.shape[0]
        x=torch.reshape(x,(zero_dim,250,2)) #1(1,250,2)
        x = F.relu(self.deconv_1(x)) #(--> 1, 250, 10)
        x = F.relu(self.deconv_2(x)) #(--> 1, 150, 50)
        x = F.relu(self.deconv_3(x)) #( --> 1, 50, 250)
        x = F.relu(self.deconv_4(x)) #(--> 1, 25, 500)
        x = F.relu(self.deconv_5(x)) #(--> 1, 10, 2500)
        x = torch.tanh(self.deconv_6(x)) #(1, 7, 5000)
        x=x.squeeze()
        return x

model_7to7_syn=pTOP_7to7().to(device)
model_7to7_norm=pTOP_7to7().to(device)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

class Transpose1dLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding=11, upsample=None, output_padding=1):
        super(Transpose1dLayer, self).__init__()
        self.upsample = upsample
        self.upsample_layer = torch.nn.Upsample(scale_factor=upsample)
        reflection_pad = kernel_size // 2
        self.reflection_pad = nn.ConstantPad1d(reflection_pad, value=0)
        self.conv1d = torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride)
        self.Conv1dTrans = nn.ConvTranspose1d(in_channels, out_channels, kernel_size, stride, padding, output_padding)

    def forward(self, x):
        if self.upsample:
            #x = torch.cat((x, in_feature), 1)
            return self.conv1d(self.reflection_pad(self.upsample_layer(x)))
        else:
            return self.Conv1dTrans(x)


class pTOP_1to7(nn.Module):
    def __init__(self,latent_dim=100, post_proc_filt_len=512,upsample=True):
        super(pTOP_1to7, self).__init__()
        # "Dense" is the same meaning as fully connection.
        stride = 4
        if upsample:
            stride = 1
            upsample = 5
        # if upsample is anything but none Transpose1dLayer will do
        # self.conv1d(self.reflection_pad(self.upsample_layer(x)))
        # which is a 1d convolution on padded and upsampled data x
        self.deconv_1 = Transpose1dLayer(250 , 250, 25, stride, upsample=upsample)
        self.deconv_2 = Transpose1dLayer(250, 150, 25, stride, upsample=upsample)
        self.deconv_3 = Transpose1dLayer(150, 50, 25, stride, upsample=upsample)
        self.deconv_4 = Transpose1dLayer( 50, 25, 25, stride, upsample=2)
        self.deconv_5 = Transpose1dLayer( 25, 10, 25, stride, upsample=upsample)
        self.deconv_6 = Transpose1dLayer(  10, 7, 25, stride, upsample=2)


        #new convolutional layers
        self.conv_1 = nn.Conv1d(1, 10, 25, stride=2, padding=25 // 2)
        self.conv_2 = nn.Conv1d(10, 25, 25, stride=5, padding= 25 // 2)
        self.conv_3 = nn.Conv1d(25, 50 , 25, stride=2, padding= 25 // 2)
        self.conv_4 = nn.Conv1d(50, 150 , 25, stride=5, padding= 25 // 2)
        self.conv_5 = nn.Conv1d(150, 250 , 25, stride=5, padding= 25 // 2)
        self.conv_6 = nn.Conv1d(250, 250 , 25, stride=5, padding= 25 // 2)
        self.flatt = nn.Flatten()
        self.linear1 = nn.Linear(500,100)
        self.linear2 = nn.Linear(100,500)

        for m in self.modules():
            if isinstance(m, nn.ConvTranspose1d) or isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight.data)

    def forward(self, x, LS=False):
        self.LS=LS
        if x.ndim==2:
          x=x.unsqueeze(0)
        x = F.leaky_relu(self.conv_1(x)) #(1,1,5000 --> 1, 10, 2500)
        x = F.leaky_relu(self.conv_2(x)) #( --> 1, 25, 500)
        x = F.leaky_relu(self.conv_3(x)) #(--> 1, 50, 250)
        x = F.leaky_relu(self.conv_4(x)) # --> 1, 150, 50)
        x = F.leaky_relu(self.conv_5(x)) #(--> 1, 250, 10)
        x = F.leaky_relu(self.conv_6(x)) #(--> 1, 250, 2)-->flatten into (1,500)), then to linear ((1,100)), and then back
        x = self.flatt(x) # (1,500)
        LS = self.linear1(x) #(1,100)
        if self.LS is True:
          return LS
        x = self.linear2(LS) #(1,500)
        zero_dim=x.shape[0]
        x=torch.reshape(x,(zero_dim,250,2)) #1(1,250,2)
        x = F.relu(self.deconv_1(x)) #(--> 1, 250, 10)
        x = F.relu(self.deconv_2(x)) #(--> 1, 150, 50)
        x = F.relu(self.deconv_3(x)) #( --> 1, 50, 250)
        x = F.relu(self.deconv_4(x)) #(--> 1, 25, 500)
        x = F.relu(self.deconv_5(x)) #(--> 1, 10, 2500)
        x = torch.tanh(self.deconv_6(x)) #(1, 7, 5000)
        x=x.squeeze()
        return x

model_1to7_syn=pTOP_1to7().to(device)
model_1to7_norm=pTOP_1to7().to(device)
# PATH="/content/artifacts/Model:v3/model"
# model.load_state_dict(torch.load(PATH))

Create data to run trough the pTOP7to7 for feature extraction. A sample from syn data will be run trough trained pTOP1to7 models, the 7 lead output will be run trough a pTOP7to7 model and its latent space compared to the latent space of the real 7 leads.


In [ ]:
## get datasets
Syn_data_dir="/content/custom_data/syn_train"
Norm_data_dir="/content/custom_data/norm_train"
test_data_1to7_syn=CD_syn_1_7(Syn_data_dir,target="test")
test_data_1to7_norm=CD_normal_1_7(Norm_data_dir,target="test")
test_data_7to7_syn=CD_syn_7_7(Syn_data_dir,target="test")
test_data_7to7_norm=CD_normal_7_7(Norm_data_dir,target="test")

In [ ]:
len(test_data_7to7_norm)

In [ ]:
#set up 1to7 models for prediction
syn1_7_path="/content/artifacts/pTOP_1_7_syn/model"
model_1to7_syn.load_state_dict(torch.load(syn1_7_path, map_location=device))
norm1_7_path="/content/artifacts/pTOP_1_7_norm/model"
model_1to7_norm.load_state_dict(torch.load(norm1_7_path, map_location=device))

#set up 7to7 models
syn7_7_path="/content/artifacts/pTOP_7_7_syn/model"
model_7to7_syn.load_state_dict(torch.load(syn7_7_path, map_location=device))
norm7_7_path="/content/artifacts/pTOP_7_7_norm/model"
model_7to7_norm.load_state_dict(torch.load(norm7_7_path, map_location=device))

In [ ]:
import torch
import pandas as pd

def get_pred_12lead(dataset=None,Set=None,model=None,random=True,upscale=None):
  """
  Function takes a Tensor Dataset as input,first a random file from the dataset is selected,
  then the Tensor pair is recombined and shaped into a df-->df_Input. 
  X is used afterwards as input into the model. The predictions are safed as --> df_output.
  Both dataframes are now unscaled by 5011, the max value of the whole dataset.
  Then a tuple pair of input and output is returned.
  """
  if random:
    import random
    limit=len(dataset)
    rand_idx=random.randint(0,limit)
    X,y=dataset[rand_idx]
  if random is False:
      X,y=Set
  #need to combine tensors to make dataframe for plotting input and output side by side
  full_tensor=torch.cat((X,y.squeeze()))
  full_tensor=full_tensor.numpy()
  df_input=pd.DataFrame(full_tensor).T
  df_input.columns = ["real_I","real_II","real_v1","real_v2","real_v3","real_v4","real_v5","real_v6"]
  model.to("cpu")
  model.eval()
  with torch.inference_mode():
    output=model(X)
  output=output.detach().numpy()
  output=output.squeeze().T
  #unscale data
  df_output = pd.DataFrame(output,columns=["pred_II","pred_v1","pred_v2","pred_v3","pred_v4","pred_v5","pred_v6"])
  #calculate the 4 missing leads
  df_input.insert(2, "real_III", df_input["real_II"] - df_input["real_I"])
  df_input.insert(3,"real_aVR",0.5*(df_input["real_I"] + df_input["real_II"]))
  df_input.insert(4,"real_aVL",(df_input["real_I"] -0.5) * df_input["real_II"])
  df_input.insert(5,"real_aVF",(df_input["real_II"] -0.5) * df_input["real_I"])
  df_output.insert(0,"real_I",df_input["real_I"])
  df_output.insert(2,"real_III",df_output["pred_II"] - df_output["real_I"])
  df_output.insert(3,"real_aVR",0.5*(df_output["real_I"] + df_output["pred_II"]))
  df_output.insert(4,"real_aVL",(df_output["real_I"] -0.5) * df_output["pred_II"])
  df_output.insert(5,"real_aVF",(df_output["pred_II"] -0.5) * df_output["real_I"])
  if upscale:
    df_input=df_input*upscale
    df_output=df_output*upscale
    
  return df_input,df_output

# lead III value = (lead II value) - (lead I value)
# lead aVR value = -0.5*(lead I value + lead II value)
# lead aVL value = lead I value - 0.5 * lead II value
# lead aVF value = lead II value - 0.5 * lead I value


In [ ]:

import pandas as pd
from pathlib import Path
def plotECG_12Lead(df1=None,df2=None,title=None,pad_df2=True,path=None,createECG=True,scale=None):
  """
  takes two dataframes with identical columns, concats them and plots them as ecg using ecg_plot
  it also takes the first column of df1 and ads it to df1 if pad_df2 is True
  """
  import ecg_plot
  index=["real_I","real_II","real_III","real_aVR","real_aVL","real_aVF","real_v1","real_v2","real_v3","real_v4","real_v5","real_v6","real_I",
         "pred_II","pred_III","pred_aVR","pred_aVL","pred_aVF","pred_v1","pred_v2","pred_v3","pred_v4","pred_v5","pred_v6"]
  if createECG==True:
    ecg_path=path
    if Path(ecg_path).is_dir():
        print(f"{ecg_path} directory exists.")
    else:
        print(f"Did not find {ecg_path} directory, creating one...")
        Path(ecg_path).mkdir(parents=True, exist_ok=False)
  if scale:
    frames=[df1/1000,df2/1000]
  if scale is None:
    frames=[df1,df2]
  combined_df=pd.concat(frames,axis=1,join="outer",)
  if createECG is True:
    ecg_plot.plot(combined_df.values.T, sample_rate = 500,title = title,
                      lead_index = index )
    ecg_plot.save_as_png('ecg',ecg_path+"/")
  return combined_df

  

In [ ]:
#get prediction from dataset for synthetic data, this is for testing purposes to see that the ecg looks right.
syn_df_in_1to7,syn_df_out_1to7=get_pred(dataset=test_data_1to7_syn,model=model_1to7_syn,upscale=5011)
combined_df=plotECG(syn_df_in_1to7,syn_df_out_1to7,path=str(ecg_dir),scale=True)


In [ ]:
# syn_df_in_1to7=syn_df_in_1to7/1000
# import ecg_plot
# ecg_plot.plot(syn_df_in_1to7.values.T, sample_rate = 500)
# import cv2 as cv
# from PIL import Image
# def png_to_jpg(path=None):
#   im = cv.imread("/content/main_folder/ecg/ecg.png")
#   im = Image.fromarray(im)
#   im.save("/content/main_folder/ecg/ecg.pdf")
# png_to_jpg()

In [ ]:
#get prediction from dataset for PTB normal data, this is for testing purposes to see that the ecg looks right
norm_df_in_1to7,norm_df_out_1to7=get_pred(dataset=test_data_1to7_norm,model=model_1to7_norm,upscale=33)
combined_df=plotECG(norm_df_in_1to7,norm_df_out_1to7,path=str(ecg_dir),scale=None)


In [ ]:
#get ecgs from 20 random sets
lenght=len(test_data_1to7_norm)
range=list(np.arange(lenght))
import random
indexes = random.sample(range,21)
for k,idx in enumerate(indexes):
  x,y=test_data_1to7_norm[idx]
  Set=(x,y)
  df_in,df_out=get_pred(model=model_1to7_norm,Set=Set,random=False,upscale=33)
  combined_df=plotECG(df_in,df_out,title=f"ecg{k+1}",scale=None,path="/content/ecg_files")

In [ ]:
#get prediction from dataset for synthetic data, this is for testing purposes to see that the ecg looks right.
syn_df_in_7to7,syn_df_out_7to7=get_pred_FE(dataset=test_data_7to7_syn,model=model_7to7_syn,upscale=5011)
combined_df=plotECG_FE(syn_df_in_7to7,syn_df_out_7to7,path=str(ecg_dir),scale=True)

In [ ]:
#get prediction from dataset for synthetic data, this is for testing purposes to see that the ecg looks right.
norm_df_in_7to7,norm_df_out_7to7=get_pred_FE(dataset=test_data_7to7_norm,model=model_7to7_norm,upscale=33)
combined_df=plotECG_FE(norm_df_in_7to7,norm_df_out_7to7,path=str(ecg_dir))

In [ ]:
## getting the latent spaces for real and predicted data
# first for synthetic data
#first run real data trough the 7t7syn model adn extract latent spaces, 
#then run predicted data trough 7to7syn model and extract latent spaces,
#then compare
## look at latent space
### get latent spaces for real data
test_loader_7to7_syn=ml(test_data_7to7_syn,32)
Latent_spaces_real = torch.empty((1,100), dtype=torch.float32)
for x in tqdm(test_loader_7to7_syn):
  model_7to7_syn.to(device)
  model_7to7_syn.eval()
  with torch.inference_mode():
    x=x.to(device)
    output=model_7to7_syn(x,LS=True)
    Latent_spaces_real = torch.cat((Latent_spaces_real, output.detach().cpu()), 0)
# Latent_spaces=Latent_spaces.numpy()
Latent_spaces_real.shape

In [ ]:
# get latent spaces for predicted data
test_loader_1to7_syn=ml(test_data_1to7_syn,32)
Latent_spaces_predicted = torch.empty((1,100), dtype=torch.float32)
for (x,y) in tqdm(test_loader_1to7_syn):
  model_1to7_syn.to(device)
  model_7to7_syn.to(device)
  model_1to7_syn.eval()
  model_7to7_syn.eval()
  with torch.inference_mode():
    x=x.to(device)
    output=model_1to7_syn(x)
    output=model_7to7_syn(output,LS=True)
    Latent_spaces_predicted = torch.cat((Latent_spaces_predicted, output.detach().cpu()), 0)
# Latent_spaces=Latent_spaces.numpy()
Latent_spaces_predicted.shape

In [ ]:
!pip install umap-learn
!pip install umap-learn[plot]

In [ ]:
Latent_spaces_predicted.shape

In [ ]:
##use UMAP to compare latent spaces
## using UMAP
import umap.umap_ as umap
from sklearn.datasets import load_digits
embedding_real = umap.UMAP(n_neighbors=7,
                      min_dist=0.3,
                      n_components=2,
                      random_state=42,
                      metric='correlation').fit(Latent_spaces_real.data)

embedding_predicted = umap.UMAP(n_neighbors=7,
                      min_dist=0.3,
                      n_components=2,
                      random_state=42,
                      metric='correlation').fit(Latent_spaces_predicted.data)


In [ ]:
df=pd.DataFrame(embedding_real)

In [ ]:
import umap.plot
umap.plot.points(embedding_real)

In [ ]:
import umap.plot
umap.plot.points(embedding_predicted)

In [ ]:
import torch
import pandas as pd

def get_pred_FE(dataset=None,Set=None,model=None,random=True,upscale=None):
  """
  Function takes a Tensor Dataset as input,first a random file from the dataset is selected,
  then the Tensor pair is recombined and shaped into a df-->df_Input. 
  X is used afterwards as input into the model. The predictions are safed as --> df_output.
  Both dataframes are now unscaled by 5011, the max value of the whole dataset.
  Then a tuple pair of input and output is returned.
  """
  if random:
    import random
    limit=len(dataset)
    rand_idx=random.randint(0,limit)
    X=dataset[rand_idx]
  if random is False:
      X=Set
  #need to combine tensors to make dataframe for plotting input and output side by side
  # full_tensor=torch.cat((X,y.squeeze()))
  # full_tensor=full_tensor.numpy()
  input=X.detach().numpy()
  input=input.squeeze().T
  df_input=pd.DataFrame(input,columns = ["real_II","real_v1","real_v2","real_v3","real_v4","real_v5","real_v6"])
  model.to("cpu")
  model.eval()
  with torch.inference_mode():
    output=model(X)
  output=output.detach().numpy()
  output=output.squeeze().T
  #unscale data
  df_output = pd.DataFrame(output,columns=["pred_II","pred_v1","pred_v2","pred_v3","pred_v4","pred_v5","pred_v6"])
  if upscale:
    df_input=df_input*upscale
    df_output=df_output*upscale
  return df_input,df_output

In [ ]:

import pandas as pd
from pathlib import Path
def plotECG_FE(df1=None,df2=None,title=None,pad_df2=True,path=None,createECG=True,scale=None):
  """
  takes two dataframes with identical columns, concats them and plots them as ecg using ecg_plot
  it also takes the first column of df1 and ads it to df1 if pad_df2 is True
  """
  index=["real_II","real_v1","real_v2","real_v3","real_v4","real_v5","real_v6",
         "pred_II","pred_v1","pred_v2","pred_v3","pred_v4","pred_v5","pred_v6"]
  if createECG==True:
    ecg_path=path
    if Path(ecg_path).is_dir():
        print(f"{ecg_path} directory exists.")
    else:
        print(f"Did not find {ecg_path} directory, creating one...")
        Path(ecg_path).mkdir(parents=True, exist_ok=True)
    import ecg_plot
  if scale:
    frames=[df1/1000,df2/1000]
  if scale is None:
    frames=[df1,df2]
  combined_df=pd.concat(frames,axis=1,join="outer",)
  if createECG is True:
    ecg_plot.plot(combined_df.values.T, sample_rate = 500,title = title,
                      lead_index = index )
    ecg_plot.save_as_png('ecg',ecg_path+"/")
  return combined_df

  

In [ ]:
# dataset=Custom_dataset("/content/main_folder/train_dir")
# df_in,df_out=get_pred_FE(model=model,dataset=dataset,upscale=5011)


In [ ]:
# combined_df=plotECG_FE(df_in,df_out,scale=True,path=str(ecg_dir))